#### Notebook to compute timeseries of ROI from the LEMON fMRI measurements
Use the timeseries to compute connectivity between the ROI

In [3]:
import os
import pandas as pd
import numpy as np

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=2,style='whitegrid')

In [9]:
#point to the relevant directory containing LEMON dataset
mri_data_dir = r'../../ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/MRI_MPILMBB_LEMON/MRI_Preprocessed_Derivetives/'

In [10]:
from nilearn import datasets
from nilearn.image import load_img, resample_to_img
from nilearn.plotting import view_img

In [11]:
curr_atlas         = 'cort-maxprob-thr0-2mm'
curr_dataset       = datasets.fetch_atlas_harvard_oxford(atlas_name=curr_atlas, symmetric_split=True)
curr_atlas         = curr_dataset.maps
curr_atlas_labels  = curr_dataset.labels

In [12]:
len(curr_atlas_labels)

97

In [13]:
conn_atlas   = load_img('conn_atlas/atlas.nii')
conn_network = load_img('conn_atlas/networks.nii')

In [14]:

with open('conn_atlas/atlas.txt') as file:
    conn_atlas_name = [line.rstrip() for line in file]
    

with open('conn_atlas/networks.txt') as file:
    conn_network_name = [line.rstrip() for line in file]


In [15]:
len(conn_atlas_name), len(conn_network_name)

(132, 32)

In [16]:
conn_network.shape, conn_atlas.shape

((91, 109, 91, 32), (182, 218, 182))

In [17]:
conn_atlas_resampled = resample_to_img(conn_atlas, curr_atlas)

In [18]:
curr_atlas.shape, conn_atlas_resampled.shape

((91, 109, 91), (91, 109, 91))

In [19]:
from nilearn.maskers import NiftiLabelsMasker
from nilearn import plotting

conn_atlas_masker = NiftiLabelsMasker(labels_img=conn_atlas_resampled, standardize=True,
                           memory='nilearn_cache', verbose=5)


conn_network_masker = NiftiLabelsMasker(labels_img=conn_network, standardize=True,
                           memory='nilearn_cache', verbose=5)

In [20]:
all_subj = [x for x in os.listdir(mri_data_dir) if x.startswith('sub')]

In [21]:
conn_network.shape

(91, 109, 91, 32)

In [4]:
out_dir = 'fmri_interm'
if(not os.path.exists(out_dir)):
    os.mkdir(out_dir)

In [22]:
for idx,each_subj in enumerate(all_subj):

    with pd.HDFStore('fmri_interm/rsfmri_connAtlasConNetwork.h5') as f_pd:
       
        if('/'+each_subj in f_pd.keys()):
            continue
        
        # Here we go from nifti files to the signal time series in a numpy
        # array. Note how we give confounds to be regressed out during signal
        # extraction

        ex_fmri_file      = os.path.join(mri_data_dir,'{}/func/{}_ses-01_task-rest_acq-AP_run-01_MNI2mm.nii.gz'.format(each_subj,
                                                                                                                      each_subj))
        ex_fmri_confounds = os.path.join(mri_data_dir,'{}/func/{}_ses-01_task-rest_acq-AP_run-01_confounds.txt'.format(each_subj,
                                                                                                                      each_subj))
        
        
        try:
            time_series_lemon_connAtlas   = conn_atlas_masker.fit_transform(ex_fmri_file,
                                                                           confounds=ex_fmri_confounds)

            atlas_data_pd = pd.DataFrame(time_series_lemon_connAtlas,
                                        columns=conn_atlas_name)


            network_data = {}
            for idx_network in np.arange(conn_network.shape[-1]):
                conn_network_masker = NiftiLabelsMasker(labels_img= conn_network.slicer[:,:,:,1], standardize=True,
                           memory='nilearn_cache', verbose=5)
                time_series_lemon_connNetwork = conn_network_masker.fit_transform(ex_fmri_file,
                                                               confounds=ex_fmri_confounds)
                network_data[conn_network_name[idx_network]] = time_series_lemon_connNetwork.squeeze()

            network_data_pd = pd.DataFrame(network_data)



            fconn_pd = pd.concat([atlas_data_pd,network_data_pd],axis=1)#,columns=aal_labels[1:])

            f_pd[each_subj] = fconn_pd
        except:
            print('Error for: {}'.format(ex_fmri_file))
            continue

    print('Done: {}/{}'.format(idx,len(all_subj)))
    


[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[   2.,    0.,    0.,  -90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
Error for: ../../ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/MRI_MPILMBB_LEMON/MRI_Preprocessed_Derivetives/sub-010318/func/sub-010318_ses-01_task-rest_acq-AP_run-01_MNI2mm.nii.gz
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[   2.,    0.,    0.,  -90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
Error for: ../../ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/MRI_MPILMBB_LEMON/MRI_Preprocessed_Derivetives/sub-010316/func/sub-010316_ses-01_task-rest_acq-AP_run-01_MNI2mm.nii.gz
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[   2.,    0.,    0.,  -90.],
       [   0.,    

[Memory]0.0s, 0.0min    : Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[  -2.,    0.,    0.,   90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
[Memory]0.0s, 0.0min    : Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[  -2.,    0.,    0.,   90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
[Memory]0.0s, 0.0min    : Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[  -2.,    0.,  

/home/bl44/mambaforge/lib/python3.10/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'sub-010032'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


________________________________________________________________________________
[Memory] Calling nilearn.maskers.base_masker._filter_and_extract...
_filter_and_extract('../../ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/MRI_MPILMBB_LEMON/MRI_Preprocessed_Derivetives/sub-010287/func/sub-010287_ses-01_task-rest_acq-AP_run-01_MNI2mm.nii.gz', 
{ 'background_label': 0,
  'clean_kwargs': {},
  'detrend': False,
  'dtype': None,
  'high_pass': None,
  'high_variance_confounds': False,
  'keep_masked_labels': True,
  'labels': None,
  'labels_img': <nibabel.nifti1.Nifti1Image object at 0x7f9d36adb130>,
  'low_pass': None,
  'mask_img': None,
  'reports': True,
  'smoothing_fwhm': None,
  'standardize': True,
  'standardize_confounds': True,
  'strategy': 'mean',
  't_r': None,
  'target_affine': None,
  'target_shape': None}, confounds=[ '../../ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/MRI_MPILMBB_LEMON/MRI_Preprocessed_Derivetives/sub-010287/func/sub-010287_ses-01_task

/home/bl44/mambaforge/lib/python3.10/site-packages/joblib/memory.py:353: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  return self.func(*args, **kwargs)


______________________________________________filter_and_extract - 34.8s, 0.6min
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[  -2.,    0.,    0.,   90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
________________________________________________________________________________
[Memory] Calling nilearn.maskers.base_masker._filter_and_extract...
_filter_and_extract('../../ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/MRI_MPILMBB_LEMON/MRI_Preprocessed_Derivetives/sub-010287/func/sub-010287_ses-01_task-rest_acq-AP_run-01_MNI2mm.nii.gz', 
{ 'background_label': 0,
  'clean_kwargs': {},
  'detrend': False,
  'dtype': None,
  'high_pass': None,
  'high_variance_confounds': False,
  'keep_masked_labels': True,
  'labels': None,
  'labels_img': <nibabel.nifti1.Nifti1Image object at 0x7f9d33c70c40>,
  'low_pass': None,
  'mask_img': None,
  'reports': True,
  'smooth

[Memory]0.0s, 0.0min    : Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[  -2.,    0.,    0.,   90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
[Memory]0.0s, 0.0min    : Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[  -2.,    0.,    0.,   90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])
)
[Memory]0.0s, 0.0min    : Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.wrapped] loading data from Nifti1Image(
shape=(91, 109, 91),
affine=array([[  -2.,    0.,  

In [4]:
with pd.HDFStore('fmri_interm/rsfmri_connAtlasConNetwork.h5','r') as f_pd:
    print(len(f_pd.keys()))
    ex_data = f_pd[f_pd.keys()[0]]

212


In [5]:
ex_data

,FP r (Frontal Pole Right),FP l (Frontal Pole Left),IC r (Insular Cortex Right),IC l (Insular Cortex Left),SFG r (Superior Frontal Gyrus Right),SFG l (Superior Frontal Gyrus Left),MidFG r (Middle Frontal Gyrus Right),MidFG l (Middle Frontal Gyrus Left),"IFG tri r (Inferior Frontal Gyrus, pars triangularis Right)","IFG tri l (Inferior Frontal Gyrus, pars triangularis Left)",...,"FrontoParietal.LPFC (L) (-43,33,28)","FrontoParietal.PPC (L) (-46,-58,49)","FrontoParietal.LPFC (R) (41,38,30)","FrontoParietal.PPC (R) (52,-52,45)","Language.IFG (L) (-51,26,2)","Language.IFG (R) (54,28,1)","Language.pSTG (L) (-57,-47,15)","Language.pSTG (R) (59,-42,13)","Cerebellar.Anterior (0,-63,-30)","Cerebellar.Posterior (0,-79,-32)"
0,-1.084055,-0.899419,0.271314,0.672710,0.231296,0.456636,-0.300492,0.173952,0.402153,0.077099,...,2.332850,2.332850,2.332850,2.332850,2.332850,2.332850,2.332850,2.332850,2.332850,2.332850
1,-0.799504,-0.738438,0.311701,0.372894,0.667388,0.785591,-0.411176,0.041681,0.734046,0.106025,...,1.821773,1.821773,1.821773,1.821773,1.821773,1.821773,1.821773,1.821773,1.821773,1.821773
2,-0.524372,-0.463804,0.144761,0.190962,0.841145,0.957542,-0.393076,0.166094,0.781384,0.229785,...,1.657450,1.657450,1.657450,1.657450,1.657450,1.657450,1.657450,1.657450,1.657450,1.657450
3,-0.080059,-0.259801,0.456621,0.497621,1.079925,1.007165,-0.308445,0.116318,0.919326,0.267812,...,1.299809,1.299809,1.299809,1.299809,1.299809,1.299809,1.299809,1.299809,1.299809,1.299809
4,0.063191,-0.256009,0.590738,0.612057,1.041330,0.775416,-0.369338,-0.089987,0.759513,0.131430,...,0.844161,0.844161,0.844161,0.844161,0.844161,0.844161,0.844161,0.844161,0.844161,0.844161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,1.201400,1.097225,-0.775320,-1.151330,0.780947,1.144245,-0.026782,-0.142163,-1.928131,-0.142458,...,0.388570,0.388570,0.388570,0.388570,0.388570,0.388570,0.388570,0.388570,0.388570,0.388570
648,1.119528,1.070680,-0.873661,-1.217396,0.635996,1.144434,-0.199335,-0.172342,-2.158323,-0.291772,...,0.479038,0.479038,0.479038,0.479038,0.479038,0.479038,0.479038,0.479038,0.479038,0.479038
649,1.235067,1.213219,-0.921825,-1.049169,0.607145,1.205735,-0.018994,0.092999,-2.249132,-0.361909,...,0.795958,0.795958,0.795958,0.795958,0.795958,0.795958,0.795958,0.795958,0.795958,0.795958
650,1.333537,1.310468,-0.987955,-0.983760,0.635480,1.283394,0.065730,0.220125,-2.316348,-0.482516,...,0.977436,0.977436,0.977436,0.977436,0.977436,0.977436,0.977436,0.977436,0.977436,0.977436


In [ ]:
all_conn = []
all_upps = []
all_subj = []

with pd.HDFStore('fmri_interm/rsfmri_connAtlasConNetwork.h5') as f_pd:
    
    all_subj_keys = f_pd.keys()

    for idx,each_subj in enumerate(all_subj_keys):
        curr_subj = each_subj[1:]
        
         
        subj_pd        = f_pd[each_subj]    
        all_region     = subj_pd.columns.values
        subj_pd['ACC'] = subj_pd[src_region_acc]#.mean(axis=1)       
        
        subj_corr_pd   = subj_pd.corr(method='pearson').loc[:,np.setdiff1d(all_region,src_region_acc)].loc['ACC'].to_frame()
        
        subj_sel_corr = subj_corr_pd
        subj_sel_corr_= flatten_df(subj_sel_corr)
        
        all_conn.append(subj_sel_corr_)
        all_subj.append(curr_subj)
        
        
        print('Completed: {}/{}'.format(idx,len(all_subj_keys)))

In [ ]:
all_conn_pd         = pd.concat(all_conn,axis=0).reset_index(drop=True)

In [ ]:
all_conn_pd['subj'] = all_subj

In [ ]:
all_conn_pd.to_csv('../Data/mmodal_neuro_ConnAtlas_ACC.csv')